# Netflix Watch Log ETL
### This notebook performs ETL for a netflix watch log data, as well as replace personal information so that it's not exposed to the public.

In [ ]:
# Import libraries
import pandas as pd
import datetime as dt
import os
import string
import random

Collect all source files

In [ ]:
# Set root path
mainFolder = "./Netflix Watch Log/"
sourceFolder = "./Netflix Watch Log/Source/"

# Initialiize dataset
dataset = pd.DataFrame()

# group all related source files in a list to get data from all realated files
listDeviceFiles = [fn for fn in os.listdir(sourceFolder) if "Devices" in fn]
listProfileFiles = [fn for fn in os.listdir(sourceFolder) if "Profiles" in fn]
listClickstreamFiles = [fn for fn in os.listdir(sourceFolder) if "Clickstream" in fn]
listSearchHistory = [fn for fn in os.listdir(sourceFolder) if "SearchHistory" in fn]
listViewingActivity = [fn for fn in os.listdir(sourceFolder) if "ViewingActivity" in fn]

In [ ]:
# read all Profiles files
datalist = [pd.read_csv(sourceFolder+currFile) for currFile in listProfileFiles]
dataset = pd.concat(datalist)
dataset.drop(columns=dataset.columns[1:], inplace=True)

In [ ]:
# Replace profile names with key "User" suffiixed with index of existing profile name value
listProfiles=list(dataset['Profile Name'].value_counts().index)
ProfileNameDict={profile:"User " + str(listProfiles.index(profile)) for profile in listProfiles}  # Re-usable dictionary for profile name
dataset.replace({"Profile Name": ProfileNameDict},inplace=True)
ProfileNameDict

In [ ]:
dataset

In [ ]:
dataset.to_csv(mainFolder+"AllProfiles.csv", index=False)

In [ ]:
# read all Devices files
datalist = [pd.read_csv(sourceFolder+currFile) for currFile in listDeviceFiles]
dataset = pd.concat(datalist)

In [ ]:
# Replace all Device - Profile name
dataset.replace({"Profile Name": ProfileNameDict},inplace=True)

In [ ]:
# Replace all Devices - Esn
listEsn=list(dataset['Esn'].value_counts().index)
esnDict={esn:"Esn " + str(listEsn.index(esn)) for esn in listEsn}
dataset.replace({"Esn": esnDict},inplace=True)

In [ ]:
# Replace all Devices - Device Types
listDeviceTypes=list(dataset['Device Type'].value_counts().index)
DeviceTypesDict={deviceType:"Device Type " + str(listDeviceTypes.index(deviceType)) for deviceType in listDeviceTypes}  # Re-usable dictionary for Device Types
dataset.replace({"Device Type": DeviceTypesDict},inplace=True)

In [ ]:
# Remove deactivated profiles
dataset=dataset[dataset['Deactivation Time'].isnull()]

In [ ]:
# Extrack Date & Time
dataset['First Playback Date'] = pd.to_datetime(dataset["Acct First Playback Date"]).dt.date
dataset['First Playback Time'] = pd.to_datetime(dataset["Acct First Playback Date"]).dt.time

dataset['Last Playback Date'] = pd.to_datetime(dataset["Acct Last Playback Date"]).dt.date
dataset['Last Playback Time'] = pd.to_datetime(dataset["Acct Last Playback Date"]).dt.time

In [ ]:
# ['Profile Name', 'Esn', 'Device Type', 'First Playback Date', 'First Playback Time', 'Last Playback Date', 'Last Playback Time']
dataset.drop(columns=['Acct First Playback Date',
       'Acct Last Playback Date',
       'Acct First Playback Date For User Generated Plays',
       'Acct Last Playback Date For User Generated Plays',
       'Profile First Playback Date', 'Profile Last Playback Date',
       'Profile First Playback Date For User Generated Plays',
       'Profile Last Playback Date For User Generated Plays',
       'Deactivation Time'], inplace=True)

In [ ]:
dataset

In [ ]:
dataset.to_csv(mainFolder+"AllDevices.csv", index=False)

In [ ]:
# read all Clickstream files
datalist = [pd.read_csv(sourceFolder+currFile) for currFile in listClickstreamFiles]
dataset = pd.concat(datalist)

In [ ]:
# Replace all Clickstream - Profile name
dataset.replace({"Profile Name": ProfileNameDict},inplace=True)

In [ ]:
# Replace "Source" value with key "Source" suffixed by index of the source name
listSources=list(dataset['Source'].value_counts().index)
sourceDict={source:"Source " + str(listSources.index(source)) for source in listSources}
dataset.replace({"Source": sourceDict},inplace=True)

In [ ]:
# Extrack Date & Time
dataset['Click Date'] = pd.to_datetime(dataset["Click Utc Ts"]).dt.date
dataset['Click Time'] = pd.to_datetime(dataset["Click Utc Ts"]).dt.time

In [ ]:
# Remove unuwanted columns
dataset.drop(columns=["Referrer Url", "Webpage Url", "Click Utc Ts"], inplace=True)

In [ ]:
dataset

In [ ]:
dataset.to_csv(mainFolder+"AllClickstream.csv", index=False)

In [ ]:
# read all SearchHistory files
datalist = [pd.read_csv(sourceFolder+currFile) for currFile in listSearchHistory]
dataset = pd.concat(datalist)

In [ ]:
# Replace all SearchHistory - Profile name
dataset.replace({"Profile Name": ProfileNameDict},inplace=True)

In [ ]:
# Replace all SearchHistory - Device
listDevices=list(dataset['Device'].value_counts().index)
devicesDict={device:"Device " + str(listDevices.index(device)) for device in listDevices}
dataset.replace({"Device": devicesDict},inplace=True)

In [ ]:
# Extrack Date & Time
dataset['Date Searched'] = pd.to_datetime(dataset["Utc Timestamp"]).dt.date
dataset['Time Searched'] = pd.to_datetime(dataset["Utc Timestamp"]).dt.time

In [ ]:
dataset.drop(columns=["Utc Timestamp"], inplace=True)

In [ ]:
dataset

In [ ]:
dataset.to_csv(mainFolder+"AllSearchHistory.csv", index=False)

In [ ]:
# read all ViewingActivity files
datalist = [pd.read_csv(sourceFolder+currFile) for currFile in listViewingActivity]
dataset = pd.concat(datalist)

In [ ]:
# Replace all ViewingActivity - Profile name
dataset.replace({"Profile Name": ProfileNameDict},inplace=True)

In [ ]:
# Replace all ViewingActivity - Device Type 
dataset.replace({"Device Type": DeviceTypesDict},inplace=True)

In [ ]:
# Extrack Date & Time
dataset['Date Viewed'] = pd.to_datetime(dataset["Start Time"]).dt.date
dataset['Time Viewed'] = pd.to_datetime(dataset["Start Time"]).dt.time

# Conevrt to common unit seconds
dataset['Duration in Seconds'] = pd.to_timedelta(dataset["Duration"]).dt.total_seconds()
dataset['Bookmared in Seconds'] = pd.to_timedelta(dataset["Bookmark"]).dt.total_seconds()

In [ ]:
dataset.drop(columns=["Duration", "Bookmark", "Latest Bookmark", "Start Time"], inplace=True)

In [ ]:
dataset

In [ ]:
dataset.to_csv(mainFolder+"AllViewingActivity.csv", index=False)